# Evaluate Models

In [1]:
import leaderbot as lb
import numpy as np
import pickle

## Load Data with 100% Traing (no test)

This is used to generate tables D.1 and D.2, bit not D.3.

In [8]:
# Load form file instead
filename = '../benchmark/models_train_full.pkl'
# filename = '../benchmark/models_train_full_both_ties.pkl'
with open(filename, 'rb') as f:
    results = pickle.load(f)

models = results['models']
proc_time = np.array(results['proc_time'])
wall_time = np.array(results['wall_time'])
print(results['device'])
print(results['num_proc'])

x86_64
128


# Process Time (Table D.1 in paper)

In the unit of seconds.

In [3]:
for i in range(proc_time.size):
    # print(f'{proc_time[i]:>7.1f}  {wall_time[i]:>7.1f} ')
    print(f'{wall_time[i]:>7.1f} ')

    2.3 
    3.8 
   34.1 
    0.0 
    1.7 
   27.5 
    5.8 
    6.9 
  208.1 
  396.9 
    8.4 
    7.5 
  293.7 
  664.9 
   36.0 
   36.9 
  363.5 
  817.3 
    6.0 
   10.5 
  253.2 
  602.8 
    8.7 
    8.3 
  286.9 
  665.1 
   34.6 
   35.8 
  366.4 
  804.9 


# Model Selection (Table D.1 in paper)

In [4]:
# Model selection
metrics = lb.evaluate.model_selection(models, train=False, tie=False, report=True)

+----+-----------------------+---------+--------+--------------------------------+---------+---------+
|    |                       |         |        |               CEL              |         |         |
| id | model                 | # param |    NLL |    all     win    loss     tie |     AIC |     BIC |
+----+-----------------------+---------+--------+--------------------------------+---------+---------+
|  1 | BradleyTerry          |     129 | 0.6554 | 0.6553  0.3177  0.3376     inf |   256.7 |  1049.7 |
|  2 | BradleyTerryScaled    |     258 | 0.6552 | 0.6551  0.3180  0.3371     inf |   514.7 |  2100.8 |
|  3 | BradleyTerryFactor    |     645 | 0.6549 | 0.6548  0.3178  0.3370     inf |  1288.7 |  5253.9 |
|  4 | BradleyTerry          |     129 | 0.6351 | 0.6351  0.3056  0.3295     inf |   256.7 |  1049.8 |
|  5 | BradleyTerryScaled    |     258 | 0.6346 | 0.6346  0.3059  0.3287     inf |   514.7 |  2100.8 |
|  6 | BradleyTerryFactor    |     645 | 0.6342 | 0.6342  0.3057  0.3285 

# Model Fit (Table D.2 in paper)

In [6]:
# Goodness of fit. This evaluates how well the model fits training data.
metrics = lb.evaluate.goodness_of_fit(models, train=False, tie=False, metric='RMSE', 
                                      density=False, report=True)

+----+-----------------------+----------------------------+------+------+
|    |                       |            RMSE            |      |      |
| id | model                 |   win   loss    tie    all | KLD% | JSD% |
+----+-----------------------+----------------------------+------+------+
|  1 | BradleyTerry          |  29.7   29.7  -----   29.7 | 1.49 | 0.44 |
|  2 | BradleyTerryScaled    |  26.2   26.2  -----   26.2 | 1.42 | 0.42 |
|  3 | BradleyTerryFactor    |  17.4   17.4  -----   17.4 | 1.30 | 0.39 |
|  4 | BradleyTerry          |  35.1   35.1  -----   35.1 | 1.82 | 0.52 |
|  5 | BradleyTerryScaled    |  31.5   31.5  -----   31.5 | 1.71 | 0.49 |
|  6 | BradleyTerryFactor    |  17.3   17.3  -----   17.3 | 1.58 | 0.46 |
|  7 | RaoKupper             |  48.2   69.9  103.5   77.3 | 3.32 | 0.92 |
|  8 | RaoKupper             |  46.4   67.8   99.2   74.3 | 3.45 | 0.91 |
|  9 | RaoKupper             |  34.1   34.2   23.1   30.9 | 2.63 | 0.73 |
| 10 | RaoKupper             |  34.3  

# Generalization (Table D.3 in paper)

For this, load data with 90% trainign and 10% split.

## Load 90% training / 10% tets split data

In [9]:
# Load form file instead
filename = '../benchmark/models_train_split.pkl'
# filename = '../benchmark/models_train_split_both_ties.pkl'
with open(filename, 'rb') as f:
    results = pickle.load(f)

models = results['models']
test_data = results['test_data']
test_data_no_tie = results['test_data_no_tie']
print(results['device'])
print(results['num_proc'])

x86_64
128


### NOTE (do not read rows 4-6):

In the following, where we used ``test_data=test_data``, use the table data for only rows 1-3, and 7-30. Do not read the followning table data for rows 4-6. See the one table after this to properly generate data for rows 4-6.

In [11]:
# Generalization. This evaluates how well the model predicts test data.
metrics = lb.evaluate.generalization(models, test_data=test_data, train=False,
                                     tie=False, metric='RMSE', density=False,
                                     report=True)

+----+-----------------------+----------------------------+------+------+
|    |                       |            RMSE            |      |      |
| id | model                 |   win   loss    tie    all | KLD% | JSD% |
+----+-----------------------+----------------------------+------+------+
|  1 | BradleyTerry          | 248.6  231.7  477.4  338.3 |  inf | 7.10 |
|  2 | BradleyTerryScaled    | 246.0  234.5  477.4  338.4 |  inf | 7.09 |
|  3 | BradleyTerryFactor    | 240.3  240.2  477.4  338.3 |  inf | 7.11 |
|  4 | BradleyTerry          | 232.8  256.2  477.4  340.5 |  inf | 7.06 |
|  5 | BradleyTerryScaled    | 230.5  260.9  477.4  341.1 |  inf | 7.06 |
|  6 | BradleyTerryFactor    | 223.5  268.6  477.4  341.6 |  inf | 7.11 |
|  7 | RaoKupper             |  54.5   29.1   67.4   52.8 | 3.16 | 0.88 |
|  8 | RaoKupper             |  49.9   41.9   74.3   57.0 | 3.31 | 0.87 |
|  9 | RaoKupper             |  26.3   38.6   32.1   32.7 | 3.17 | 0.85 |
| 10 | RaoKupper             |  29.0  

### NOTE (read only rows 4-6):

In the following, where we used ``test_data=test_data_no_tie``, use the table data to read only rows 4-6, and discard all other rows. For all other rows, read the table in the above.

In [12]:
# Generalization. This evaluates how well the model predicts test data.
metrics = lb.evaluate.generalization(models, test_data=test_data_no_tie, train=False,
                                     tie=False, metric='RMSE', density=False,
                                     report=True)

+----+-----------------------+----------------------------+------+------+
|    |                       |            RMSE            |      |      |
| id | model                 |   win   loss    tie    all | KLD% | JSD% |
+----+-----------------------+----------------------------+------+------+
|  1 | BradleyTerry          |  54.7   54.7  -----   54.7 | 1.86 | 0.52 |
|  2 | BradleyTerryScaled    |  52.6   52.6  -----   52.6 | 1.84 | 0.51 |
|  3 | BradleyTerryFactor    |  52.8   52.8  -----   52.8 | 1.94 | 0.54 |
|  4 | BradleyTerry          |  30.0   30.0  -----   30.0 | 1.74 | 0.48 |
|  5 | BradleyTerryScaled    |  30.3   30.3  -----   30.3 | 1.77 | 0.48 |
|  6 | BradleyTerryFactor    |  30.4   30.4  -----   30.4 | 2.06 | 0.54 |
|  7 | RaoKupper             | 156.6  197.2  338.5  243.6 | 24.06 | 7.91 |
|  8 | RaoKupper             | 163.7  201.3  351.1  252.0 | 21.41 | 7.06 |
|  9 | RaoKupper             | 165.0  218.2  370.1  265.7 | 22.13 | 7.28 |
| 10 | RaoKupper             | 168.